In [21]:
import pandas as pd
import numpy as np
import re

#conda install -c conda-forge spacy==2.3.1
#pip install spacy[lookups]
#kernel restart

import spacy
from spacy.lang.tr import Turkish
nlp = Turkish()

import requests
from bs4 import BeautifulSoup

### Beautifulsoup Wikipedia Parsing

In [22]:
#Veri Bilimi Vikipedi sayfasını getiriyoruz
veri_bilimi = requests.get('https://tr.wikipedia.org/wiki/Veri_bilimi').content
veri_madenciliği = requests.get('https://tr.wikipedia.org/wiki/Veri_madenciliği').content

#BeautifulSoup ile html sayfasını parse ediyoruz.
vb_soup = BeautifulSoup( veri_bilimi,'html.parser' )
vd_soup = BeautifulSoup( veri_madenciliği,'html.parser' )

# HTML tag'lerinden arındırılmış şekilde sayfanın sadece ilk paragrafını geiriyoruz
VB_first_sentence = vb_soup.find_all('p')[1].get_text()
VM_first_sentence = vd_soup.find_all('p')[1].get_text()

In [23]:
print(VB_first_sentence)
print(VM_first_sentence)

Veri bilimi, yapılandırılmış ve yapılandırılmamış verilerden bilgi ve öngörü elde etmek için bilimsel yöntemleri, süreçleri, algoritmaları ve sistemleri kullanan çok disiplinli bir alandır.[1] Veri bilimi veri madenciliği ve büyük verilerle ilişkilidir.

Veri madenciliği, büyük ölçekli veriler arasından faydalı bilgiye ulaşma, bilgiyi madenleme işidir. Büyük veri yığınları içerisinden gelecekle ilgili tahminde bulunabilmemizi sağlayabilecek bağıntıların bilgisayar programı kullanarak aranması olarak da tanımlanabilir.[1][2]



In [24]:
def cleaning_text(text):
    text_nopunct = re.sub(r'[^\w\s]','', str(text).lower())
    text_nonumbers =re.sub('[0-9]+', '', str(text_nopunct))
    return text_nonumbers

In [25]:
VB =cleaning_text(VB_first_sentence)
VM =cleaning_text(VM_first_sentence)

In [26]:
print(VB)
print(VM)

veri bilimi yapılandırılmış ve yapılandırılmamış verilerden bilgi ve öngörü elde etmek için bilimsel yöntemleri süreçleri algoritmaları ve sistemleri kullanan çok disiplinli bir alandır veri bilimi veri madenciliği ve büyük verilerle ilişkilidir

veri madenciliği büyük ölçekli veriler arasından faydalı bilgiye ulaşma bilgiyi madenleme işidir büyük veri yığınları içerisinden gelecekle ilgili tahminde bulunabilmemizi sağlayabilecek bağıntıların bilgisayar programı kullanarak aranması olarak da tanımlanabilir



# Spacy Tokenization

### Word Tokenization

In [27]:
#"nlp" Object is used to create documents with linguistic annotations.
nlp_VB = nlp(VB)
nlp_VM = nlp(VM)

In [28]:
def tokenization(nlp_doc):
    token_list = []
    for token in nlp_doc:
        token_list.append(token.text)
    return token_list

In [29]:
nlp_VB = tokenization(nlp_VB)
print(nlp_VB)

['veri', 'bilimi', 'yapılandırılmış', 've', 'yapılandırılmamış', 'verilerden', 'bilgi', 've', 'öngörü', 'elde', 'etmek', 'için', 'bilimsel', 'yöntemleri', 'süreçleri', 'algoritmaları', 've', 'sistemleri', 'kullanan', 'çok', 'disiplinli', 'bir', 'alandır', 'veri', 'bilimi', 'veri', 'madenciliği', 've', 'büyük', 'verilerle', 'ilişkilidir', '\n']


In [30]:
nlp_VM = tokenization(nlp_VM)
print(nlp_VM)

['veri', 'madenciliği', 'büyük', 'ölçekli', 'veriler', 'arasından', 'faydalı', 'bilgiye', 'ulaşma', 'bilgiyi', 'madenleme', 'işidir', 'büyük', 'veri', 'yığınları', 'içerisinden', 'gelecekle', 'ilgili', 'tahminde', 'bulunabilmemizi', 'sağlayabilecek', 'bağıntıların', 'bilgisayar', 'programı', 'kullanarak', 'aranması', 'olarak', 'da', 'tanımlanabilir', '\n']


### Sentence Tokenization

In [31]:
text = """ Veri bilimi, yapılandırılmış ve yapılandırılmamış verilerden bilgi ve öngörü elde etmek için bilimsel yöntemleri, süreçleri, algoritmaları ve sistemleri kullanan çok disiplinli bir alandır. 
Veri bilimi veri madenciliği ve büyük verilerle ilişkilidir.
"""

In [32]:
# Create the pipeline 'sentencizer' component
sentencizer = nlp.create_pipe('sentencizer')

# Add the component to the pipeline
nlp.add_pipe(sentencizer)

#  "nlp" Object is used to create documents with linguistic annotations.
nlp_doc = nlp(text)

# create list of sentence tokens
sents_list = []
for sent in nlp_doc.sents:
    sents_list.append(sent.text)
print(sents_list)


[' Veri bilimi, yapılandırılmış ve yapılandırılmamış verilerden bilgi ve öngörü elde etmek için bilimsel yöntemleri, süreçleri, algoritmaları ve sistemleri kullanan çok disiplinli bir alandır.', '\nVeri bilimi veri madenciliği ve büyük verilerle ilişkilidir.', '\n']


# Spacy Stop Words

In [35]:
#importing stop words from English language.
import spacy
spacy_stopwords = spacy.lang.tr.stop_words.STOP_WORDS

#Printing the total number of stop words:
print('Number of stop words: %d' % len(spacy_stopwords))

#Printing first ten stop words:
print('First 10 stop words: %s' % list(spacy_stopwords)[:20])

Number of stop words: 551
First 10 stop words: ['çokluk', 'nere', 'neyse', 'sahiden', 'birlikte', 'ait', 'enikonu', 'emme', 'gene', 'olduğu', 'derken', 'zarfında', 'az', 'tüm', 'ama', 'çabuk', 'yenilerde', 'başkası', 'kimse', 'oluyor']


In [36]:
doc1 = vb_soup.find_all('p')[1].get_text()
text = cleaning_text(text)

from spacy.lang.en.stop_words import STOP_WORDS


filtered_sent=[]

#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(text)

# filtering stop words
for word in doc:
    if word.is_stop==False:
        filtered_sent.append(word)
print("Filtered Sentence:",filtered_sent)

Filtered Sentence: [ , veri, bilimi, yapılandırılmış, yapılandırılmamış, verilerden, bilgi, öngörü, elde, etmek, bilimsel, yöntemleri, süreçleri, algoritmaları, sistemleri, kullanan, disiplinli, bir, alandır, 
, veri, bilimi, veri, madenciliği, büyük, verilerle, ilişkilidir, 
]


In [37]:
doc1

'Veri bilimi, yapılandırılmış ve yapılandırılmamış verilerden bilgi ve öngörü elde etmek için bilimsel yöntemleri, süreçleri, algoritmaları ve sistemleri kullanan çok disiplinli bir alandır.[1] Veri bilimi veri madenciliği ve büyük verilerle ilişkilidir.\n'

# Spacy Lemmatization

In [41]:
doc1 = vb_soup.find_all('p')[1].get_text()
text = cleaning_text(text)

# Implementing lemmatization
lem = nlp(text)
# finding lemma for each word
for word in lem:
    print(word.text,word.lemma_)

   
veri veri
bilimi bilim
yapılandırılmış yapı
ve ve
yapılandırılmamış yapı
verilerden veri
bilgi bilgi
ve ve
öngörü öngörü
elde el
etmek et
için için
bilimsel bilimsel
yöntemleri yöntem
süreçleri süreç
algoritmaları algoritma
ve ve
sistemleri sistem
kullanan kullan
çok çok
disiplinli disiplin
bir bir
alandır al

 

veri veri
bilimi bilim
veri veri
madenciliği madencilik
ve ve
büyük büyük
verilerle veri
ilişkilidir ilişki

 

